# Import thư viện

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import pandas as pd
import time
import re

d:\TLCN\Source Code\GPT test\gptenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Url seed

In [2]:
url_seed = [
    # "https://www.dieutri.vn/chandoandieutri",
    # "https://www.dieutri.vn/benhly",
    # "https://www.dieutri.vn/sach",
    # "https://www.dieutri.vn/duoc",
    "https://www.dieutri.vn/thuocgoc",
    "https://www.dieutri.vn/xetnghiem",
    "https://www.dieutri.vn/phacdo",
    "https://www.dieutri.vn/thongtin"
]

url_skip = [
    "https://www.dieutri.vn/",
    "https://www.dieutri.vn/chandoandieutri",
    "https://www.dieutri.vn/benhly",
    "https://www.dieutri.vn/sach",
    "https://www.dieutri.vn/duoc",
    "https://www.dieutri.vn/thuocgoc",
    "https://www.dieutri.vn/xetnghiem",
    "https://www.dieutri.vn/phacdo",
    "https://www.dieutri.vn/thongtin",
    "https://www.dieutri.vn/",
    "https://www.dmca.com/Protection/Status.aspx?ID=b8d58b76-e5dd-4c75-bf41-0f0bebc705cd&refurl=https://www.dieutri.vn/",
    "https://www.dieutri.vn/gioithieu",
    "https://www.dieutri.vn/lienhe"
]

# Url này thêm khi lỗi cấu trúc
url_more = [
    "https://www.dieutri.vn/tta",
    "https://www.dieutri.vn/ttb",
    "https://www.dieutri.vn/ttc",
    "https://www.dieutri.vn/ttd",
    "https://www.dieutri.vn/ttf",
    "https://www.dieutri.vn/ttg",
    "https://www.dieutri.vn/tth",
    "https://www.dieutri.vn/ttm",
    "https://www.dieutri.vn/ttn",
    "https://www.dieutri.vn/tto",
    "https://www.dieutri.vn/ttp",
    "https://www.dieutri.vn/ttr",
    "https://www.dieutri.vn/tts",
    "https://www.dieutri.vn/ttt",
    "https://www.dieutri.vn/ttv",
    "https://www.dieutri.vn/ttx",
    "https://www.dieutri.vn/tty",
    "https://www.dieutri.vn/bgmiendichdiung",
    "https://www.dieutri.vn/ttk",
    "https://www.dieutri.vn/ttl",
    "https://www.dieutri.vn/ttz",
    "https://www.dieutri.vn/vandeduoc"
    
]

# Crawling

In [3]:
data_dict = {
    "subject": [],
    "headline1": [],
    "headline2": [],
    "text": [],
    "url_source": []
}

In [4]:
def get_html(url):
    while True:
        try:
            # Gửi yêu cầu HTTP để lấy nội dung HTML
            response = requests.get(url)
            
            if response.status_code == 200:
                html_content = response.text
                return html_content
        except:
            print(f"Unable to connect to {url}! Try again in 3 seconds")
            time.sleep(3)
            
def data_to_csv(data):
    df = pd.DataFrame(data)
    df.to_csv("dieutri_data_crawling2.csv", index=False)
    
def crawling():
    for i, url in enumerate(url_seed):
        html_content_seed = get_html(url)
        soup = BeautifulSoup(html_content_seed, 'html.parser')
        subject = soup.find_all('h1')[0].text
        print(f"Starting with URL_SEED {i+1}: {subject}")
        links = soup.find_all('a')
        for link in links:
            href = link.get('href')
            if (href not in url_skip) or href in url_more:
                html_content1 = get_html(href)
                soup1 = BeautifulSoup(html_content1, 'html.parser')
                headline1 = soup1.find_all('h1')[0].text
                process_links = []
                for link1 in soup1.find_all('a'):
                    href1 = link1.get('href')
                    if href1 not in url_skip and href in href1 and href1 != href and href1 not in url_more:
                        process_links.append(href1)
                for link1 in tqdm(process_links, desc=f"Analysing {headline1}: "):
                    html_content2 = get_html(link1)
                    soup2 = BeautifulSoup(html_content2, 'html.parser')
                    try:
                        headline2 = soup2.find_all('h1')[0].text
                        detail = soup2.select('.detail')[0].text
                        tacgia = soup2.select('.tacgia')[0].text
                        text = re.sub(r'\n+', '\n', detail.replace(tacgia, "").strip())
                        url_source = link1
                        data_dict["subject"].append(subject)
                        data_dict["headline1"].append(headline1)
                        data_dict["headline2"].append(headline2)
                        data_dict["text"].append(text)
                        data_dict["url_source"].append(url_source)
                    except:
                        print(f"Error with {link1}! Pass!")
                        continue
                data_to_csv(data_dict)
                print("SAVED!")

In [5]:
crawling()

Starting with URL_SEED 1: Thuốc A - Z


Analysing Thuốc gốc và biệt dược theo vần A: 100%|██████████| 50/50 [00:29<00:00,  1.71it/s]


SAVED!


Analysing Thuốc gốc và biệt dược theo vần B: 100%|██████████| 50/50 [00:42<00:00,  1.18it/s]


SAVED!


...

SAVED!


Analysing Giảm bạch cầu trung tính: phân tích triệu chứng: : 0it [00:00, ?it/s]


SAVED!


Analysing Tốc độ máu lắng và Protein phản ứng C: phân tích triệu chứng: : 0it [00:00, ?it/s]


SAVED!
